## Corporate Model Simulation: Quantifying Contributions under CSL

This notebook models **how corporations contribute to open-source projects under CSL-like licenses**, both through cash contributions and verified labor. It allows us to explore **trade-offs between corporate adoption, community funds, and developer engagement**.

---

### Model Overview

We simulate an ecosystem with three agent types:

- **Individuals** – many, small contributors.
- **Small Organizations** – moderate adoption and revenue impact.
- **Large Organizations** – few, high-revenue adopters; subject to CSL rules on contribution and barriers.

Key parameters include:

- **`contributor_share_pct`** – portion of revenue redirected to the contributor fund.
- **`corporate_cash_pct`** – direct monetary contribution by large organizations.
- **`labor_credit_pct`** – fraction of verified corporate labor credited to the fund.
- **`barrier_to_entry`** – restrictions limiting adoption by large organizations.
- **`fork_penalty`** – discourages organizations from bypassing contribution requirements.
- **`verif_rate`** – fraction of labor contribution that is verifiable/usable.
- **`steer_factor`** – portion of labor not contributing to public goods (internal effort).

We track **over time**:

- **Contributor Fund** – total resources available to support developers.
- **Cash from Corporations** – raw monetary contributions.
- **Effective Corporate Labor** – value of labor contributing to public goods.
- **Net Cash to Fund** – cash after labor credit is applied.
- **Total Adoption** – number of adopters per timestep.
- **Active Developers** – engaged contributors.
- **Total Revenue** – overall revenue from adopters.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(2)

# -----------------------------
# Agent definitions
# -----------------------------
agents = {
    'Individuals': {'count':500, 'base_revenue':1.0, 'base_adopt_prob':0.01},
    'SmallOrgs': {'count':50, 'base_revenue':50.0, 'base_adopt_prob':0.05},
    'LargeOrgs': {'count':5, 'base_revenue':2000.0, 'base_adopt_prob':0.2},
}
T = 120

# -----------------------------
# Simulation function
# -----------------------------
def simulate_with_corporate_model(params, T_steps=None, seed=2):
    if T_steps is None: T_steps=T
    np.random.seed(seed)
    contributor_fund = params.get('initial_fund', 1000.0)
    active_devs = 50.0
    metrics = {'t':[], 'contributor_fund':[], 'net_cash_to_fund':[], 'adopter_share':[],
               'labor_value':[], 'total_adoption':[], 'active_developers':[]}
    
    R_per_large = params.get('corp_avg_adoption_revenue_per_large',5000.0)
    H_per_large = params.get('corp_labor_hours_per_large',10.0)
    
    for t in range(T_steps):
        adoption=0.0; revenue=0.0; effort=0.0; adopter_share_total=0.0
        for ag_name, ag in agents.items():
            count, base_rev, adopt_prob = ag['count'], ag['base_revenue'], ag['base_adopt_prob']
            if ag_name == 'LargeOrgs':
                adopt_prob *= params.get('large_adopt_multiplier',1.0)*(1-params.get('barrier_to_entry',0))
                forks = np.random.binomial(count, params.get('fork_penalty',0)*0.5)
                adopters = np.random.binomial(count-forks, min(0.999, adopt_prob))
            else:
                adopters = np.random.binomial(count, min(0.999, adopt_prob + params.get('network_effect',0.0)*(active_devs/100.0)))
            
            rev = adopters*base_rev
            effort += adopters*params.get('contributor_incentive',0.1)
            adoption += adopters; revenue += rev
            
            # Adopter share
            adopter_share = adopters*params.get('adopter_share_pct',0.0)*base_rev
            contributor_fund += adopter_share
            adopter_share_total += adopter_share

        # Corporate cash + labor
        n_large = agents['LargeOrgs']['count']
        corporate_cash = params.get('corporate_cash_pct',0.0) * (R_per_large*n_large)
        H_corp = H_per_large * n_large
        labor_hour_value = params.get('labor_hour_value',120.0)
        effective_labor_value = H_corp * labor_hour_value * params.get('labor_weight',0.6) * params.get('verif_rate',0.8) * (1-params.get('steer_factor',0.2))
        labor_credit_amount = min(effective_labor_value*params.get('labor_credit_pct',0.5), corporate_cash)
        net_cash = max(0.0, corporate_cash - labor_credit_amount)
        contributor_fund += net_cash

        # Payouts and enforcement
        enforcement_cost = contributor_fund * params.get('enforcement_cost_pct',0.0)
        contributor_fund -= enforcement_cost
        payout = min(contributor_fund*params.get('payout_rate',0.015), effort*params.get('payout_effort_scale',0.05))
        contributor_fund -= payout

        # Active developer update
        active_devs += 0.1*(adoption/10.0)+0.2*(payout/100.0)+0.0005*(effective_labor_value+params.get('volunteer_hours_per_timestep',100.0)*labor_hour_value)-0.05*(enforcement_cost/100.0)
        active_devs = max(1.0, active_devs)

        # Store metrics
        metrics['t'].append(t)
        metrics['contributor_fund'].append(contributor_fund)
        metrics['net_cash_to_fund'].append(net_cash)
        metrics['adopter_share'].append(adopter_share_total)
        metrics['labor_value'].append(effective_labor_value)
        metrics['total_adoption'].append(adoption)
        metrics['active_developers'].append(active_devs)

    return pd.DataFrame(metrics)

# -----------------------------
# CSL parameters
# -----------------------------
params = {'large_adopt_multiplier':0.3,'contributor_share_pct':0.10,'barrier_to_entry':0.3,'contributor_incentive':0.3,
          'enforcement_cost_pct':0.02,'fork_penalty':0.6,'network_effect':0.005,'adopter_share_pct':0.01,
          'corporate_cash_pct':0.05,'corp_avg_adoption_revenue_per_large':20000.0,'corp_labor_hours_per_large':20.0,
          'labor_hour_value':150.0,'labor_weight':0.6,'labor_credit_pct':0.5,'verif_rate':0.7,'steer_factor':0.2,
          'initial_fund':5000.0,'payout_rate':0.015,'payout_effort_scale':0.05,'volunteer_hours_per_timestep':150.0}

df = simulate_with_corporate_model(params)

# -----------------------------
# Grid-style plotting with stacked fund bar
# -----------------------------
fig, axes = plt.subplots(2, 2, figsize=(16,10))
fig.suptitle('CSL Contributor Fund & Corporate Contributions', fontsize=18, fontweight='bold')

# 1) Stacked Contributor Fund Breakdown
axes[0,0].bar(df['t'], df['net_cash_to_fund'], label='Net Cash Contribution', color='tab:orange')
axes[0,0].bar(df['t'], df['adopter_share'], bottom=df['net_cash_to_fund'], label='Adopter Share', color='tab:blue')
axes[0,0].plot(df['t'], df['contributor_fund'], label='Total Fund', color='black', linewidth=2)
axes[0,0].set_title('Contributor Fund Breakdown')
axes[0,0].set_xlabel('Time'); axes[0,0].set_ylabel('USD')
axes[0,0].legend(); axes[0,0].grid(True)

# 2) Corporate Labor Value
axes[0,1].plot(df['t'], df['labor_value'], label='Effective Corporate Labor', color='tab:green')
axes[0,1].set_title('Corporate Labor Contributions')
axes[0,1].set_xlabel('Time'); axes[0,1].set_ylabel('USD Equivalent')
axes[0,1].legend(); axes[0,1].grid(True)

# 3) Active Developers
axes[1,0].plot(df['t'], df['active_developers'], color='tab:purple')
axes[1,0].set_title('Active Developer Engagement')
axes[1,0].set_xlabel('Time'); axes[1,0].set_ylabel('Number of Developers')
axes[1,0].grid(True)

# 4) Total Adoption
axes[1,1].plot(df['t'], df['total_adoption'], color='tab:brown')
axes[1,1].set_title('Project Adoption Over Time')
axes[1,1].set_xlabel('Time'); axes[1,1].set_ylabel('Adopters')
axes[1,1].grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


# Contributor Fund vs Net Cash – CSL Corporate Model

This section explains how the **Contributor Fund** works in the CSL corporate model simulation, breaking down the mechanics, purpose, and visualization of cash and labor contributions.

---

## 1. Contributor Fund

The **Contributor Fund** is the **total pool of resources available to reward contributors**. It accumulates from multiple sources:

- **Initial Fund**: A starting cash amount (seed fund) to jumpstart the project.
- **Adopter Share**: Percentage of revenue from individuals or small organizations adopting the project (`adopter_share_pct`).
- **Corporate Contributions**: Cash from large organizations after accounting for labor credit.
- **Adjustments**: Deducts payouts to contributors, enforcement costs, or other governance expenses.

> **Analogy:** Think of it as the project's "bank account" for paying contributors.

---

## 2. Net Cash Contribution

**Net Cash Contribution** represents **the actual cash entering the Contributor Fund from corporations**, after adjusting for labor contributions:

- **Corporate Cash**: A fraction of corporate revenue is pledged to the fund (`corporate_cash_pct`).  
- **Labor Credit**: Corporations can provide verified OSS labor instead of or in addition to cash. Labor has a monetary equivalent:

\[
\text{Labor Value} = \text{Hours} \times \text{Hourly Rate} \times \text{Weight} \times \text{Verification Rate} \times (1-\text{Steering Factor})
\]

- **Net Cash**: The fund receives only the remaining cash after accounting for labor credits:

\[
\text{Net Cash} = \text{Corporate Cash} - \text{Labor Credit Amount}
\]

> **Analogy:** Corporations contribute to the project, but if some contribution is in labor, only the leftover cash flows into the fund.

---

## 3. Relationship Between Contributor Fund and Net Cash

\[
\text{Contributor Fund}_{t} = \text{Contributor Fund}_{t-1} + \text{Adopter Share} + \text{Net Cash} - \text{Payouts} - \text{Enforcement Costs}
\]

- **Contributor Fund** = the total “wallet”  
- **Net Cash** = just the cash component from corporations  
- The fund typically grows faster than net cash alone due to **adopter shares** and accumulated contributions.

---

## 4. How Contributor Fund Could Work in Reality

1. **Funding Sources**
   - GitHub Sponsors, Open Collective, direct donations (PayPal/Stripe)
   - Corporate sponsorships or license-mandated revenue share

2. **Governance**
   - License rules or project governance determine:
     - How payouts are calculated (fixed % of fund, performance-based, etc.)  
     - How labor contributions are verified and valued

3. **Labor Credit**
   - Employee OSS hours can count as a contribution:
     - Verified hours × labor value × weight × verification rate × adjustment factors

4. **Payout Mechanism**
   - Funds are distributed to contributors via:
     - Manual payments  
     - Automated processes (GitHub Sponsors API, Open Collective, or smart contracts)

---

## 5. Simulation Example Flow

| Step | Fund Effect |
|------|------------|
| T=0  | Initial fund = $5,000 |
| T=1  | Adopter share adds $50 |
| T=1  | Corporate cash = $5,000; labor credit = $2,500 → Net cash = $2,500 |
| T=1  | Payouts = $200 → Fund decreases |
| T=1  | Enforcement cost = $50 → Fund decreases |
| T=1  | Total Contributor Fund = previous + net cash + adopter share - payouts - enforcement cost |

---

## 6. Key Takeaways for Engineers

- **Contributor Fund** is the **total “wallet”** available for contributors.
- **Net Cash** is a **subset of the fund from corporations**, adjusted for labor contributions.
- **Labor Credits** allow the project to recognize corporate OSS work as a contribution.
- License governance decides **how contributions are collected and distributed**.
- Visualization of fund vs net cash helps understand **cash vs labor contribution dynamics**.

---

## 7. Visualization in the Notebook

In the 05_corporate_model notebook, we show a **grid of four charts** to explain the model:

| Chart | Purpose |
|-------|---------|
| Contributor Fund vs Net Cash | Total fund growth vs cash from corporations after labor credit |
| Corporate Labor Contributions | Effective corporate labor vs total public labor value |
| Active Developer Engagement | Number of contributors over time |
| Project Adoption Over Time | Adoption dynamics across all agent types |



> **Goal:** Make it clear **where the money comes from, who is contributing, and how it affects active developers and adoption**.
